In [18]:
%matploblib inline

UsageError: Line magic function `%matploblib` not found.


In [19]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [20]:
device = ''
gpu_num = 0
if torch.cuda.is_available():
    device = 'cuda:'+str(gpu_num)
else:
    device = 'cpu'

print('now using '+ device +' as device')

now using cuda:0 as device


In [21]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        """
        xを入力に受け取り順伝播する
        """
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [22]:
# NeuralNetworkクラスのインスタンスを作成し変数device上に移動させる
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [23]:
x = torch.rand(1, 28, 28, device=device)
logits = model(x)
# nn.Softmaxは関数を返す高階関数 func(logits) -> nn.Softmax(dim=1)(logits)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(y_pred)

tensor([6], device='cuda:0')


In [24]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flattenレイヤーで，2次元の画像を1次元の値へと平坦化する．
ミニバッチの0次元目は，サンプル番号を示す次元で，この次元はnn.Flattenを通しても変化しない．

In [25]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linearは，線形変換を施す．
linear layerは重みとバイアスのパラメータを保持している．

In [26]:
layer_1 = nn.Linear(in_features=28*28, out_features=20)
hidden_1 = layer_1(flat_image)
print(hidden_1.size())

torch.Size([3, 20])


nn.ReLUは，非線形な活性化関数である．ニューラルネットワークの入力と出力の間にある，複雑な関係性を表現するために重要な要素である．

In [28]:
print('before ReLU: ', hidden_1, end='\n\n')
hidden_1 = nn.ReLU()(hidden_1)
print('after ReLU: ', hidden_1)

before ReLU:  tensor([[0.0000, 0.3416, 0.5311, 0.0000, 0.0000, 0.2162, 0.3235, 0.1535, 0.0086,
         0.0000, 0.0000, 0.4195, 0.2369, 0.7379, 0.0000, 0.0000, 0.0000, 0.0253,
         0.0000, 0.0000],
        [0.0683, 0.1876, 0.4520, 0.0000, 0.0000, 0.0000, 0.1567, 0.2630, 0.0000,
         0.0191, 0.0000, 0.3495, 0.0000, 0.3337, 0.0000, 0.0000, 0.0000, 0.2326,
         0.0000, 0.3852],
        [0.0689, 0.0206, 0.4425, 0.0151, 0.0000, 0.0341, 0.2844, 0.3195, 0.0000,
         0.0000, 0.0000, 0.3026, 0.0000, 0.2739, 0.0000, 0.2519, 0.0000, 0.0531,
         0.0000, 0.1635]], grad_fn=<ReluBackward0>)

after ReLU:  tensor([[0.0000, 0.3416, 0.5311, 0.0000, 0.0000, 0.2162, 0.3235, 0.1535, 0.0086,
         0.0000, 0.0000, 0.4195, 0.2369, 0.7379, 0.0000, 0.0000, 0.0000, 0.0253,
         0.0000, 0.0000],
        [0.0683, 0.1876, 0.4520, 0.0000, 0.0000, 0.0000, 0.1567, 0.2630, 0.0000,
         0.0191, 0.0000, 0.3495, 0.0000, 0.3337, 0.0000, 0.0000, 0.0000, 0.2326,
         0.0000, 0.3852],
      

nn.Sequentialは，モジュールを順番に格納する箱のような要素．
入力データはnn.Sequentialに定義された順番に各モジュールを伝播する．

In [29]:
seq_modules = nn.Sequential(
    flatten,
    layer_1,
    nn.ReLU(),
    nn.Linear(20,10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.3315,  0.1550, -0.2051,  0.2970,  0.0321, -0.0154, -0.1014,  0.1262,
          0.0711,  0.0094],
        [ 0.2871,  0.3391, -0.0714,  0.6016,  0.1445, -0.0132, -0.0574,  0.2134,
          0.0287, -0.1351],
        [ 0.2126,  0.1111, -0.2029,  0.2579,  0.1028,  0.0447, -0.0281,  0.0561,
          0.1066,  0.1559]], grad_fn=<AddmmBackward0>)


In [30]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1283, 0.1075, 0.0750, 0.1239, 0.0951, 0.0907, 0.0832, 0.1045, 0.0989,
         0.0930],
        [0.1137, 0.1198, 0.0794, 0.1557, 0.0986, 0.0842, 0.0806, 0.1056, 0.0878,
         0.0745],
        [0.1132, 0.1022, 0.0747, 0.1184, 0.1014, 0.0957, 0.0890, 0.0968, 0.1018,
         0.1069]], grad_fn=<SoftmaxBackward0>)


In [31]:
print('model structure: \n', model)

for name, param in model.named_parameters():
    print('layer:', name, 'size:', param.size(), 'values:', param[:2], '\n')

model structure: 
 NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)
layer: linear_relu_stack.0.weight size: torch.Size([512, 784]) values: tensor([[ 0.0179,  0.0166, -0.0159,  ...,  0.0169, -0.0252, -0.0181],
        [ 0.0240,  0.0246, -0.0318,  ..., -0.0096, -0.0201, -0.0207]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

layer: linear_relu_stack.0.bias size: torch.Size([512]) values: tensor([-0.0064,  0.0027], device='cuda:0', grad_fn=<SliceBackward0>) 

layer: linear_relu_stack.2.weight size: torch.Size([512, 512]) values: tensor([[-0.0399, -0.0402,  0.0180,  ..., -0.0305,  0.0341, -0.0034],
        [-0.0294,  0.0104,  0.0415,  ...,  0.0300, -0.0159,  0.0054]],
       device='cuda:0', grad_fn=<S